In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.head().T


In [ ]:
# Fazendo um cópia do dataframe
df_raw = df.copy()

In [ ]:
# Verificar o dataframe
df.info()

In [ ]:
# Corrigindo a coluna TotalCharges e convertendo em FLOAT
df['TotalCharges'] = df['TotalCharges'].str.replace(' ','-1').astype(float)

In [ ]:
df.info()

# Preparar os dados


In [ ]:
# Transformando dados categorias em números
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:

# Importandoa função
from sklearn.model_selection import train_test_split

In [ ]:
# Separação do dataframe em treino e teste
train, test = train_test_split(df, test_size=0.2,random_state=42)

In [ ]:
# Separando o dataframe de treino em treino e validação
train, valid = train_test_split(train,test_size=0.2,random_state=42)

In [ ]:
train.shape, valid.shape, test.shape

In [ ]:
# Selecionar as colunas para treinamento
feats = [c for c in df.columns if c not in['customerID','Churn']]

In [ ]:
# Importar o RandomForest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instanciar o RandomForest
rf = RandomForestClassifier(n_estimators=200, min_samples_split=5,max_depth=4,random_state=42)

In [ ]:
# Treinamento do modelo
rf.fit(train[feats],train['Churn'])

In [ ]:
preds = rf.predict(valid[feats])

In [ ]:
# Avaliando o desempenho do modelo
from sklearn.metrics import accuracy_score

accuracy_score(valid['Churn'], preds)

In [ ]:
df['Churn'].value_counts(normalize=True)

In [ ]:
(valid['Churn']== 0).mean()

In [ ]:
test['Churn'].value_counts(normalize=True)

In [ ]:
# Rodar o modelo para prever o Churn dos dados de teste
accuracy_score(test['Churn'], rf.predict(test[feats]))

In [ ]:
# Analisando a importância das características
pd.Series(rf.feature_importances_,index=feats).sort_values().plot.barh()

In [ ]:
# Matriz de confusao
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Churn'],rf.predict(test[feats]))

In [ ]:
pd.DataFrame(confusion_matrix(test['Churn'],rf.predict(test[feats])))

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
viz = ConfusionMatrix(RandomForestClassifier())
viz.fit(train[feats],train['Churn'])
viz.score(test[feats],test['Churn'])
viz.poof()